In [1]:
import json
import pandas
from pandas import read_csv
import pickle

In [12]:
behavioral = read_csv('results/predictions/behavioral_randomforest.tsv', delimiter = '\t',header=0,index_col=0)
diagnosis = read_csv('results/predictions/diagnosis_randomforest.tsv', delimiter = '\t',header=0,index_col=0)
prevention = read_csv('results/predictions/prevention_randomforest.tsv', delimiter = '\t',header=0,index_col=0)
treatment = read_csv('results/predictions/treatment_randomforest.tsv', delimiter = '\t',header=0,index_col=0)

In [13]:
behavioral['behavioral_prediction'].replace('not behavioral','', inplace=True)
diagnosis['diagnosis_prediction'].replace('not diagnosis','', inplace=True)
prevention['prevention_prediction'].replace('not prevention','', inplace=True)
treatment['treatment_prediction'].replace('not treatment','', inplace=True)
print(treatment.head(n=2))

           _id treatment_prediction
0  NCT04423614            treatment
1  NCT04377776                     


In [50]:
topicCategories = behavioral.merge(diagnosis.merge(prevention.merge(treatment,on='_id',how='outer'),on='_id',how='outer'),on='_id',how='outer')
topicCategories['topicCategory'] = 'XZ'+topicCategories[['behavioral_prediction',
                                                    'diagnosis_prediction',
                                                    'prevention_prediction',
                                                    'treatment_prediction']].apply(lambda x: ','.join(x), axis=1)+'XY'
has_cats = topicCategories.loc[topicCategories['topicCategory']!='XZ,,,XY'].copy()
has_cats['topicCategory'] = has_cats['topicCategory'].str.replace(',,,',',')
has_cats['topicCategory'] = has_cats['topicCategory'].str.replace(',,',',')
has_cats['topicCategory'] = has_cats['topicCategory'].str.replace('XZ,','XZ')
has_cats['topicCategory'] = has_cats['topicCategory'].str.replace(',XY','XY')
has_cats['topicCategory'] = has_cats['topicCategory'].str.replace('XZ','[')
has_cats['topicCategory'] = has_cats['topicCategory'].str.replace('XY',']')

has_cats.drop(columns=['behavioral_prediction','diagnosis_prediction','prevention_prediction','treatment_prediction'],inplace=True)

has_cats.to_csv('results/ct_topic.tsv',sep='\t',header=True)